# A3. Raw API
Source: MLB Stats API <br>

Description: 
This reads in plate appearance data from the MLB Stats API <br>
On a daily basis, it will take the current year's file and update it with data since the last update  <br>
It can also create the entire dataset from scratch

### Imports

In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import regex as re

import statsapi
from statsapi import get

baseball_path = r"C:\Users\james\Documents\MLB\Data"

### Box Score

In [2]:
# Creates box score variables
def create_box(gamePk):
    # Read in boxscore as json
    box = pd.json_normalize(statsapi.boxscore_data(gamePk, timecode=None), record_path='gameBoxInfo')
    # Take value where label = variable of interest
    try:
        weather = box.loc[box['label']=="Weather", "value"].item() 
    except:
        weather = "75 degrees, Clear."
    try:
        wind = box.loc[box['label']=="Wind", "value"].item()
    except:
        wind = "0 mph, L To R."
    try:
        venue = box.loc[box['label']=="Venue", "value"].item()
    except:
        venue = "Missing Park."
    # Date should be last label
    try:
        date = box.iloc[-1,box.columns.get_loc('label')]
    except:
        date = "November 30, 1993"
    
    return weather, wind, venue, date

### Game Events

In [3]:
# Create dataframe of game events
# Observations are runners, including the batter
def create_game(gamePk):
    list_of_lists = []
    # Read in game data
    game = statsapi.get('game_playByPlay', {'gamePk':gamePk})
    # Loop over all play
    for i in game['allPlays']:
        atBatIndex = i['about']['atBatIndex']
        inning = i['about']['inning']
        halfInning = i['about']['halfInning']
        
        outs = i['count']['outs']
        
        try:
            type = i['result']['type']
        except:
            type = None
        try:
            event = i['result']['event']
        except:
            event = None
        try:
            eventType = i['result']['eventType']
        except:
            eventType = None
        try:
            description = i['result']['description']
        except:
            description = None
        try:
            rbi = i['result']['rbi']
        except:
            rbi = 0
        try:
            awayScore = i['result']['awayScore']
        except:
            awayScore = 0
        try:
            homeScore = i['result']['homeScore']
        except:
            homeScore = 0
        
        try:
            batter = i['matchup']['batter']['id']
        except:
            batter = 999999
        try:
            batterName = i['matchup']['batter']['fullName']
        except:
            batterName = "Missing Name"
        try:
            batSide = i['matchup']['batSide']['code']
        except:
            batSide = "R"
        try:
            pitcher = i['matchup']['pitcher']['id']
        except:
            pitcher = 999999
        try:
            pitcherName = i['matchup']['pitcher']['fullName']
        except:
            pitcherName = "Missing Name"
        try:
            pitchHand = i['matchup']['pitchHand']['code']
        except:
            pitchHand = "R"
        
        try:
            postOnFirst = i['matchup']['postOnFirst']['id']
        except:
            postOnFirst = None
        try:
            postOnSecond = i['matchup']['postOnSecond']['id']
        except:
            postOnSecond = None
        try:
            postOnThird = i['matchup']['postOnThird']['id']
        except:
            postOnThird = None
        
        # Loop over runner data
        for j in i['runners']:
            id = j['details']['runner']['id']
            start = j['movement']['start']
            end = j['movement']['end']
            movementReason = j['details']['movementReason']

            list = [atBatIndex, inning, halfInning, outs, type, id, event, eventType, description, 
                    rbi, awayScore, homeScore, 
                    batter, batterName, batSide, pitcher, pitcherName, pitchHand, 
                    postOnFirst, postOnSecond, postOnThird,
                    start, end, movementReason]
            list_of_lists.append(list)
    
    df = pd.DataFrame(list_of_lists, columns=['atBatIndex', 'inning', 'halfInning', 'outs', 'type', 'id', 'event', 'eventType', 'description', 
                                              'rbi', 'awayScore', 'homeScore', 
                                              'batter', 'batterName', 'batSide', 'pitcher', 'pitcherName', 'pitchHand', 
                                              'postOnFirst', 'postOnSecond', 'postOnThird',
                                              'start', 'end', 'movementReason'])
    
    
    
    # Statcast Data 
    game = statsapi.get('game', {'gamePk':gamePk})
    list_of_lists = []
    for i in game['liveData']['plays']['allPlays']:
        atBatIndex = i['about']['atBatIndex']
        # Pitch Start Speeds
        startSpeeds = re.findall(r"startSpeed': (\d+.\d)", str(i['playEvents']))
        startSpeeds = [float(x) for x in startSpeeds]
        # Spin Rates
        spinRates = re.findall(r"spinRate': (\d+.\d)", str(i['playEvents']))
        spinRates = [float(x) for x in spinRates]
        # code (pitch type)
        code = re.findall(r"'type': {'code': '(\w+)'", str(i['playEvents']))


        ### Batter Data (will only have one value per AB at most)
        # Launch Speeds
        launchSpeeds = re.findall(r"launchSpeed': (\d+.\d)", str(i['playEvents']))
        launchSpeeds = [float(x) for x in launchSpeeds]
        # Launch Angle
        launchAngles = re.findall(r"launchAngle': (\d+.\d)", str(i['playEvents']))
        launchAngles = [float(x) for x in launchAngles]
        # Total Distances
        totalDistances = re.findall(r"totalDistance': (\d+.\d)", str(i['playEvents']))
        totalDistances = [float(x) for x in totalDistances]
        # Trajectories
        trajectories = re.findall(r"trajectory': '(\w+)'", str(i['playEvents']))
        # Hardness
        hardness = re.findall(r"hardness': '(\w+)'", str(i['playEvents']))
        # Coordinates (batted ball) (X, Y)
        coord = re.findall(r"coord\w': (\d+.\d)", str(i['playEvents']))
        coord = [float(x) for x in coord]

        statcast_list = [atBatIndex, startSpeeds, spinRates, code, launchSpeeds, launchAngles, totalDistances, trajectories, hardness, coord]
        list_of_lists.append(statcast_list)

        statcast_df = pd.DataFrame(list_of_lists, columns=['atBatIndex', 'startSpeeds', 'spinRates', 'code', 'launchSpeeds', 'launchAngles', 
                                                           'totalDistances', 'trajectories', 'hardness', 'coord'])

    
    try:
        
        df = df.merge(statcast_df, on='atBatIndex', how='left')
        # print("Merged: " + str(gamePk))
    except:
        print("No statcast df: " + str(gamePk))
    
    
    # Create variables
    df['gamePk'] = gamePk
    # Get information from box score and create variables
    weather, wind, venue, date = create_box(gamePk)
    df['weather'] = weather
    df['wind'] = wind
    df['venue'] = venue
    df['date'] = date
    
     
    return df

### Dataset

In [4]:
# Creates dataset of all games between dates
# Note1: Only do one year at a time - gets weird otherwise 
# Note2: There may be duplicate game_id values. To avoid counting twice, I make a list of ids and only create game df if I haven't already
def create_dataset(start_date, end_date):
    # List of all game dfs
    play_list = []
    # Delete existing dataframes (shouldn't really happen anymore)
    try:
        del play_df
    except:
        pass
    # Pick days
    games = statsapi.schedule(start_date=start_date,end_date=end_date)
    # List of all game_id values that have already been done
    game_list = []
    # Loop over games
    for x in games:
        # Check if this game has already been done, and if not,
        if x['game_id'] not in game_list:
            # Add its gamePk to the list of games that have been done
            game_list.append(x['game_id'])
            
            # Create game df
            game_df = create_game(x['game_id'])
            
            # Add information from the schedule
            game_df['away_name'] = x['away_name']
            game_df['home_name'] = x['home_name']
            game_df['game_date'] = x['game_date']
            game_df['venue_id'] = x['venue_id']
            game_df['game_type'] = x['game_type']
            game_df['gamePk'] = x['game_id']

            # Add it to list of dataframes
            play_list.append(game_df)
            
        # If the game has already been done, skip
        else:
            pass
    
    # Create dataframe from all games
    play_df = pd.concat(play_list, axis=0)
    
    return play_df

### Update

In [5]:
# Takes existing dataframe and adds missing dates
def update_current_year(year):
    # Read in dataset to update
    old_play_df = pd.read_csv(os.path.join(baseball_path, "A3. Raw API", "Play" + str(year) + ".csv"))

    # Convert game dates to date format
    old_play_df['game_date'] = pd.to_datetime(old_play_df['game_date'])
    # Find the last date in the sample
    last_date = old_play_df['game_date'].max()
    print("Last date in sample: " + str(last_date))
    # Add a day to it
    start_date = last_date + datetime.timedelta(days=1)
    # Only keep the date (not the time)
    start_date = start_date.date()
    start_date = str(start_date.strftime("%m/%d/%Y"))
    print("Start date in scrape: " + start_date)
    # Determine today's date
    yesterday = datetime.date.today()
    # Subtract a day
    yesterday = yesterday - datetime.timedelta(days=1)
    yesterday = str(yesterday.strftime("%m/%d/%Y"))
    # Make it a string
    print("End date in scrape: " + yesterday)
    
    new_play_df = create_dataset(start_date, yesterday)
    
    play_df = pd.concat([old_play_df, new_play_df], axis=0)
    
    play_df.drop(columns={'index'}, inplace=True)
    play_df.reset_index(inplace=True)
    
    play_df = play_df.loc[:,~play_df.columns.str.startswith('Unnamed')]
    
    play_df.to_csv(os.path.join(baseball_path, "A3. Raw API", "Play" + str(year) + ".csv"))

In [6]:
# # Run this if you want to update the existing year
update_current_year(2023)

Last date in sample: 2023-06-11 00:00:00
Start date in scrape: 06/12/2023
End date in scrape: 06/11/2023


HTTPError: 400 Client Error: Bad Request for url: https://statsapi.mlb.com/api/v1/schedule?startDate=06/12/2023&endDate=06/11/2023&sportId=1&hydrate=decisions,probablePitcher(note),linescore,broadcasts,game(content(media(epg)))

### From Scratch

In [ ]:
# # This creates complete year datasets
# # Use this when recreating a year or multiple years from scratch
# def create_complete_datasets(start_year, end_year):
#     for year in range(start_year, end_year):
#         print(year)
#         # Choose start and end dates
#         start_date = "03/29/" + str(year)
#         end_date = "11/15/" + str(year)
#         # Create dataset
#         play_df = create_dataset(start_date, end_date)
#         play_df.reset_index(inplace=True)

#         # Save as csv
#         play_df.to_csv(os.path.join(baseball_path, "A3. Raw API", "Play" + str(year) + ".csv"), index=False)

# # Run this if you want to create the datasets from scratch
# # It may be good to do this every once in a while, at least for the current year. Sometimes, games will be missing when I update (No statcast df: )
# # Not really sure what's going on there, but if they get added later, recreating the year could help
# create_complete_datasets(2019, 2023)

Notes: <br>
Date ranges seem to be problematic when you go between years. Can only do one year at a time <br> 
There may be duplicate games in a year. Not sure why. It's corrected by making a list of unique games and keeping them, but just good to keep in mind. <br>

Wishlist: <br>
May want to add more stats: launch angles, launch rate, babip, estimated_woba_using_speedangle, iso_value, etc...
https://baseballsavant.mlb.com/csv-docs

In [ ]:
print("Code was last run on: {} at {}.".format(datetime.date.today(), datetime.datetime.now().strftime("%H:%M:%S")))